# ДЗ - построение ассоциативных правил для датасета :
Можно использовать например apriori, association_rules, fpgrowth из mlxtend.frequent_patterns (работают чуть быстрее).
Написать выводы. 

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import re
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder
from mpl_toolkits.mplot3d import Axes3D
import networkx as nx

basket1 = pd.read_csv("dataset.csv", header=None)
basket1

In [13]:
!pip install mlxtend

In [151]:
basket1.shape

(1499, 14)

In [152]:
#Проверяем есть ли na строки
basket1.isna().any()

0     False
1     False
2     False
3     False
4     False
5      True
6      True
7      True
8      True
9      True
10     True
11     True
12     True
13     True
dtype: bool

In [153]:
# Удаляем строки с na
basket1 = basket1.dropna()
basket1

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,pork,sandwich bags,lunch meat,all- purpose,flour,soda,butter,vegetables,beef,aluminum foil,all- purpose,dinner rolls,shampoo,all- purpose
1,shampoo,hand soap,waffles,vegetables,cheeses,mixes,milk,sandwich bags,laundry detergent,dishwashing liquid/detergent,waffles,individual meals,hand soap,vegetables
2,pork,soap,ice cream,toilet paper,dinner rolls,hand soap,spaghetti sauce,milk,ketchup,sandwich loaves,poultry,toilet paper,ice cream,ketchup
4,pasta,tortillas,mixes,hand soap,toilet paper,vegetables,vegetables,paper towels,vegetables,flour,vegetables,pork,poultry,eggs
5,toilet paper,eggs,toilet paper,vegetables,bagels,dishwashing liquid/detergent,cereals,paper towels,laundry detergent,butter,cereals,bagels,paper towels,shampoo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1492,butter,hand soap,bagels,soap,laundry detergent,toilet paper,hand soap,fruits,ketchup,poultry,ice cream,juice,tortillas,ice cream
1494,beef,sandwich bags,hand soap,paper towels,paper towels,all- purpose,beef,fruits,coffee/tea,beef,shampoo,mixes,cheeses,juice
1495,dinner rolls,lunch meat,spaghetti sauce,pasta,vegetables,cereals,dinner rolls,soap,milk,eggs,poultry,cheeses,cereals,mixes
1496,lunch meat,eggs,poultry,vegetables,tortillas,beef,beef,individual meals,dishwashing liquid/detergent,shampoo,dishwashing liquid/detergent,dishwashing liquid/detergent,hand soap,sugar


In [154]:
#переводим в List, группируем
basket = basket1.values.tolist()
basket

[[' pork',
  ' sandwich bags',
  ' lunch meat',
  ' all- purpose',
  ' flour',
  ' soda',
  ' butter',
  ' vegetables',
  ' beef',
  ' aluminum foil',
  ' all- purpose',
  ' dinner rolls',
  ' shampoo',
  ' all- purpose'],
 [' shampoo',
  ' hand soap',
  ' waffles',
  ' vegetables',
  ' cheeses',
  ' mixes',
  ' milk',
  ' sandwich bags',
  ' laundry detergent',
  ' dishwashing liquid/detergent',
  ' waffles',
  ' individual meals',
  ' hand soap',
  ' vegetables'],
 [' pork',
  ' soap',
  ' ice cream',
  ' toilet paper',
  ' dinner rolls',
  ' hand soap',
  ' spaghetti sauce',
  ' milk',
  ' ketchup',
  ' sandwich loaves',
  ' poultry',
  ' toilet paper',
  ' ice cream',
  ' ketchup'],
 [' pasta',
  ' tortillas',
  ' mixes',
  ' hand soap',
  ' toilet paper',
  ' vegetables',
  ' vegetables',
  ' paper towels',
  ' vegetables',
  ' flour',
  ' vegetables',
  ' pork',
  ' poultry',
  ' eggs'],
 [' toilet paper',
  ' eggs',
  ' toilet paper',
  ' vegetables',
  ' bagels',
  ' dishwashin

In [155]:
basket_len = len(basket)
basket_len

1044

In [157]:
print(type(basket))

<class 'list'>


** После этого с помощью TransactionEncoder мы закодируем транзакции в формат, подходящий для функции Apriori.

In [158]:
encoder = TransactionEncoder()
transactions = pd.DataFrame(encoder.fit(basket).transform(basket), columns=encoder.columns_)
transactions

,,all- purpose,aluminum foil,bagels,beef,butter,cereals,cheeses,coffee/tea,dinner rolls,...,shampoo,soap,soda,spaghetti sauce,sugar,toilet paper,tortillas,vegetables,waffles,yogurt
0,False,True,True,False,True,True,False,False,False,True,...,True,False,True,False,False,False,False,True,False,False
1,False,False,False,False,False,False,False,True,False,False,...,True,False,False,False,False,False,False,True,True,False
2,False,False,False,False,False,False,False,False,False,True,...,False,True,False,True,False,True,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,True,True,False,False
4,False,False,False,True,False,True,True,False,False,False,...,True,False,False,False,False,True,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1039,False,False,False,True,False,True,False,False,False,False,...,False,True,False,False,False,True,True,False,False,False
1040,False,True,False,False,True,False,False,True,True,False,...,True,False,False,False,False,False,False,False,False,False
1041,False,False,False,False,False,False,True,True,False,True,...,False,True,False,True,False,False,False,True,False,False
1042,False,False,False,False,True,False,False,False,False,False,...,True,False,False,False,True,False,True,True,False,False


### Apriori и правила ассоциаций
Алгоритм Apriori будет использоваться для генерации частых наборов элементов. Мы зададим минимальную поддержку в размере 15 из общего числа транзакций. Генерируются правила ассоциаций, и мы отфильтровываем значения Lift > 1,3.

In [167]:
frequent_itemsets = apriori(transactions, min_support= 15/len(basket), use_colnames=True, max_len =3)
rules = association_rules(frequent_itemsets, metric="lift",  min_threshold = 1.3)
display(rules.head(20))
print("Rules identified: ", len(rules))

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,"( eggs, all- purpose)",( bagels),0.091954,0.315134,0.039272,0.427083,1.355243,0.010294,1.195402,0.288669
1,"( bagels, all- purpose)",( eggs),0.092912,0.309387,0.039272,0.422680,1.366187,0.010526,1.196240,0.295490
2,( eggs),"( bagels, all- purpose)",0.309387,0.092912,0.039272,0.126935,1.366187,0.010526,1.038970,0.388113
3,( bagels),"( eggs, all- purpose)",0.315134,0.091954,0.039272,0.124620,1.355243,0.010294,1.037316,0.382739
4,"( beef, all- purpose)",( fruits),0.085249,0.295019,0.035441,0.415730,1.409164,0.010291,1.206602,0.317419
5,( fruits),"( beef, all- purpose)",0.295019,0.085249,0.035441,0.120130,1.409164,0.010291,1.039643,0.411868
6,"( dinner rolls, all- purpose)",( tortillas),0.080460,0.273946,0.028736,0.357143,1.303696,0.006694,1.129417,0.253333
7,( tortillas),"( dinner rolls, all- purpose)",0.273946,0.080460,0.028736,0.104895,1.303696,0.006694,1.027299,0.320844
8,"( all- purpose, mixes)",( hand soap),0.088123,0.256705,0.030651,0.347826,1.354964,0.008030,1.139719,0.287290
9,( hand soap),"( all- purpose, mixes)",0.256705,0.088123,0.030651,0.119403,1.354964,0.008030,1.035522,0.352448


Rules identified:  252


In [168]:
rules.sort_values(by=['lift'],ascending = [False]).reset_index(drop = True).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,"( tortillas, soap)",( ketchup),0.068966,0.278736,0.029693,0.430556,1.544674,0.010470,1.266611,0.378734
1,( ketchup),"( tortillas, soap)",0.278736,0.068966,0.029693,0.106529,1.544674,0.010470,1.042042,0.488883
2,( beef),"( waffles, fruits)",0.286398,0.087165,0.037356,0.130435,1.496417,0.012392,1.049761,0.464877
3,"( waffles, fruits)",( beef),0.087165,0.286398,0.037356,0.428571,1.496417,0.012392,1.248803,0.363414
4,"( butter, coffee/tea)",( sandwich loaves),0.070881,0.281609,0.029693,0.418919,1.487590,0.009733,1.236300,0.352777
5,( sandwich loaves),"( butter, coffee/tea)",0.281609,0.070881,0.029693,0.105442,1.487590,0.009733,1.038635,0.456258
6,( ketchup),"( juice, mixes)",0.278736,0.090996,0.037356,0.134021,1.472816,0.011992,1.049683,0.445091
7,"( juice, mixes)",( ketchup),0.090996,0.278736,0.037356,0.410526,1.472816,0.011992,1.223573,0.353165
8,( fruits),"( dishwashing liquid/detergent, spaghetti sauce)",0.295019,0.080460,0.034483,0.116883,1.452690,0.010746,1.041244,0.442029
9,"( dishwashing liquid/detergent, spaghetti sauce)",( fruits),0.080460,0.295019,0.034483,0.428571,1.452690,0.010746,1.233716,0.338889


****Применение алгоритма в бизнесе
Допустим, владельцы магазина закупили слишком много cока и теперь беспокоятся, что товар пропадет, если его не удастся вовремя распродать. Что еще хуже, рентабельность продаж сока настолько низка, что они не могут позволить себе промо-скидку, не потеряв при этом большую часть прибыли.

In [172]:
juice_rules = rules[rules['consequents'].astype(str).str.contains('juice')]
juice_rules = juice_rules.sort_values(by=['lift'],ascending = [False]).reset_index(drop = True)

display(juice_rules.head(20))

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,( ketchup),"( juice, mixes)",0.278736,0.090996,0.037356,0.134021,1.472816,0.011992,1.049683,0.445091
1,"( yogurt, sandwich loaves)",( juice),0.083333,0.294061,0.035441,0.425287,1.446254,0.010936,1.228333,0.336609
2,"( ketchup, mixes)",( juice),0.089080,0.294061,0.037356,0.419355,1.426080,0.011161,1.215783,0.327995
3,( mixes),"( juice, ketchup)",0.318008,0.083333,0.037356,0.117470,1.409639,0.010856,1.038680,0.426102
4,( sandwich loaves),"( juice, pork)",0.281609,0.077586,0.030651,0.108844,1.402872,0.008802,1.035075,0.399750
5,( paper towels),"( juice, individual meals)",0.297893,0.081418,0.033525,0.112540,1.382258,0.009271,1.035069,0.393880
6,"( paper towels, spaghetti sauce)",( juice),0.078544,0.294061,0.031609,0.402439,1.368555,0.008512,1.181367,0.292257
7,"( sandwich loaves, pork)",( juice),0.076628,0.294061,0.030651,0.400000,1.360261,0.008118,1.176564,0.286826
8,( spaghetti sauce),"( juice, flour)",0.295019,0.074713,0.029693,0.100649,1.347153,0.007652,1.028839,0.365533
9,"( paper towels, shampoo)",( juice),0.077586,0.294061,0.030651,0.395062,1.343467,0.007836,1.166960,0.277161


**#** Вывод: Например, мы можем установить промо-скидку на кетчуп, йогурт, сендвичи. Некоторые ассоциации могут показаться контринтуитивными, но правила гласят, что эти продукты действительно стимулируют продажи cока.